In [16]:
import tensorflow as tf
import numpy as np
import sklearn
import random,os,io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint,CSVLogger
from tensorboard.plugins.hparams import api as hp

In [17]:
from keras.wrappers.scikit_learn import KerasRegressor

In [18]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

def build_model(hyperparams):
    hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=50)
    hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=50)
    hp_layer_3 = hp.Int('layer_3', min_value=1, max_value=1000, step=50)
    hp_layer_4 = hp.Int('layer_4', min_value=1, max_value=1000, step=50)
    hp_layer_5 = hp.Int('layer_5', min_value=1, max_value=1000, step=50)
    hp_layer_6 = hp.Int('layer_6', min_value=1, max_value=1000, step=50)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_drop_out = hp.Choice('drop_out', values=[0.1, 0.2, 0.15])
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_weights=['glorot_uniform','he_normal','he_uniform','random_uniform','random_normal']
    model = Sequential()
    model.add(layers.Input(shape=(dep_train.shape[1],)))
    model.add(layers.Dense(units=hp_layer_1, activation=hp_activation,kernel_initializer=hp_weights))
    model.add(layers.Dropout(hp_drop_out))
    model.add(layers.Dense(1,activation='linear'))

    optim=hyperparams.Choice("optimizer",["sgd","rmsprop","adam"])
    model.compile(optim, loss="mean_squared_error", metrics=["mean_squared_error"])

    return model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
model = Sequential()
model.add(layers.Input(shape=(dep_train.shape[1],)))
model.add(layers.Dense(units=12, activation='relu',kernel_initializer='random_normal'))


In [ ]:
model.add(layers.Dropout(0.3))

In [ ]:
model.add(layers.Dense(units=12, activation='linear',kernel_initializer='random_normal'))


In [ ]:
model.summary()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor

from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
import pickle
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

In [ ]:
# Create function
def nn_cl_bo2(neurons, activation, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
        
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,]
        
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        if normalization > 0.5:
            nn.add(BatchNormalization())
        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='linear'))
        nn.compile(loss='mean_squared_error', optimizer=optimizer, metrics=["mean_squared_error","mean_absolute_error","mean_absolute_percentage_error","mean_squared_logarithmic_error"])
        return nn
    es = EarlyStopping(monitor='val_loss', verbose=0, patience=20)
    nn = KerasRegressor(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, dep_train, indep_train, cv=kfold, fit_params={'callbacks':[es]}).mean()
    
    return score

In [ ]:
params_nn2 ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(20, 200),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}

# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

In [ ]:
params_nn_ = nn_bo.max['params']

learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]

params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
             'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
             'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]

params_nn_

In [ ]:
optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                'elu', 'exponential', LeakyReLU,]
    
neurons = round((10, 100))
activation = activationL[round(activation)]
optimizer = optimizerD[optimizerL[round(optimizer)]]
batch_size = round(batch_size)
epochs = round(epochs)
layers1 = round(layers1)
layers2 = round(layers2)

def nn_cl_fun():
    nn = Sequential()
    nn.add(Dense(neurons, input_dim=10, activation=activation))
    if normalization > 0.5:
        nn.add(BatchNormalization())
    for i in range(layers1):
        nn.add(Dense(neurons, activation=activation))
    if dropout > 0.5:
        nn.add(Dropout(dropout_rate, seed=123))
    for i in range(layers2):
        nn.add(Dense(neurons, activation=activation))
    nn.add(Dense(1, activation='linear'))
    nn.compile(loss='mean_squared_error', optimizer=optimizer, metrics=["mean_squared_error","mean_absolute_error","mean_absolute_percentage_error","mean_squared_logarithmic_error"])
    return nn
es = EarlyStopping(monitor='val_loss', verbose=0, patience=20)
nn = KerasRegressor(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

In [ ]:
round((10, 100))

In [19]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [33]:
def build_model(hp):
    model = keras.Sequential()
    hp_activation_2 = hp.Choice('activation_2', values=['relu', 'tanh'])
    hp_activation_3 = hp.Choice('activation_3', values=['relu', 'tanh'])
    hp_activation_4 = hp.Choice('activation_4', values=['relu', 'tanh'])
    kwargs = {'hp_activation_0':hp_activation_2,'hp_activation_1':hp_activation_3,'hp_activation_2':hp_activation_4}
    hp_weights=hp.Choice('weights', values=['glorot_uniform','he_normal','he_uniform','random_uniform','random_normal'])
    for i,r in enumerate(range(hp.Int('num_layers', 2, 4))):
        print(i,r)
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=2,
                                            max_value=10,
                                            step=2),kernel_initializer=hp_weights,
                               activation=kwargs[f'hp_activation_{i}']))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [34]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=1,
    directory='project',
    project_name='new5')

0 0
1 1


In [35]:
tuner.search(dep_train, indep_train,
             epochs=2,
             validation_data=(dep_val, indep_val))

Trial 5 Complete [00h 00m 20s]
val_mean_absolute_error: 0.0012792764464393258

Best val_mean_absolute_error So Far: 0.0006915595731697977
Total elapsed time: 00h 01m 34s
INFO:tensorflow:Oracle triggered exit


In [36]:
best_params = tuner.get_best_hyperparameters()

best_params[0].values

{'activation_2': 'tanh',
 'activation_3': 'relu',
 'activation_4': 'tanh',
 'weights': 'random_normal',
 'num_layers': 3,
 'units_0': 10,
 'units_1': 8,
 'learning_rate': 0.0001,
 'units_2': 8}

In [11]:
import yaml,sys
with io.open('variable1.yaml', 'w', encoding='utf8') as outfile:
    yaml.dump(best_params[0].values, outfile, default_flow_style=False, allow_unicode=True)
print("file_name_ids_inside form:",best_params[0].values,file=sys.stderr)

file_name_ids_inside form: {'activation': 'relu', 'weights': 'random_normal', 'num_layers': 3, 'units_0': 8, 'units_1': 6, 'learning_rate': 0.001, 'units_2': 2}
